In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import glob
import numpy as np
import joblib
import uuid
import seaborn as sns

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model, Model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten
import imutils

In [ ]:
# re-size all the images to this
SIZE = 240

train_path = '/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition-20221108T101958Z-001/Masked Face Recognition/MainDataset'
# valid_path = '/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/database/Test'
DetectorLabels = ['Mask','No Mask']


In [ ]:
# Training data and labels into respective lists
train_images = []
train_labels = []
training_data = []
prev=0

for directory_path in glob.glob(os.path.join(train_path, "*")):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        # print(img_path)
        img = cv2.imread(img_path)       
        img = cv2.resize(img, (SIZE,SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        training_data.append([train_images,train_labels])
    print(len(train_images)-prev)
    prev=len(train_images)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
training_data = np.array(training_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.25, random_state=45)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(train_labels)
train_labels_encoded = encoder.transform(train_labels)
encoder.fit(test_labels)
test_labels_encoded = encoder.transform(test_labels)

In [ ]:
#Inverse transform to get original label back. 
for i in range (0,12):
  print(i," ",encoder.inverse_transform([i]))

In [ ]:
train_images = np.array(train_images).reshape(-1,SIZE,SIZE,3)
train_images=np.array(train_images)
train_images.shape

In [ ]:
train_labels=np.array(train_labels)
train_labels.shape

In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

In [ ]:
protoPath = "D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\deploy.prototxt"
weightsPath = "D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel"

faceNet = cv2.dnn.readNet(protoPath, weightsPath)

In [ ]:
maskNet = load_model('D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\face_detector\\mask_detector.model')

In [ ]:
VGG_model = load_model("D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\featuresVGG.h5")
RF_model = joblib.load("D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\RFModel.joblib")

In [ ]:
def predict(img):
    if img is None:
        return 'Unknown'
    img = cv2.resize(img,(SIZE,SIZE))
    input_img = np.expand_dims(img, axis=0)
    input_img_feature=VGG_model.predict(input_img)
    prediction_RF = np.argmax(input_img_feature, axis=1)
    prediction_RF = encoder.inverse_transform([prediction_RF])  if prediction_RF<3 else None
    return str(prediction_RF)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Masked Face Recognition/MainDataset/Alia Bhatt/6889155f-5d48-11ed-9c5b-90324b084670.jpg')
# max = np.max(predict(img))
print(predict(img))
# max

In [ ]:
cap = cv2.VideoCapture(0)

while True:
	_ , frame = cap.read()

	# detect faces with mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations
	for (box, pred) in zip(locs, preds):
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY-120), (0,0,0), 2)
		img = frame[startY:endY-120,startX:endX]
		
		# cv2.imshow("cut",img)
		name = predict(img)
		if img is not None:
			cv2.putText(frame, name, (endX-20, endY+10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
  
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	if key == ord("q"):
		break

cap.release()
cv2.destroyAllWindows()

In [ ]:
featureDetector = load_model(r'face_detector\featuresVGGDetector.h5')
RFDetector = joblib.load(r'face_detector\RFModelDetector.joblib')

In [ ]:
def detect(img):
    if img is None:
        return None
    img = cv2.resize(img,(224,224))
    input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=featureDetector.predict(input_img)
    prediction_RF = RFDetector.predict(input_img_feature)[0]
    prediction = RFDetector.predict_proba(input_img_feature)
    return DetectorLabels[prediction_RF],prediction

In [ ]:
img = cv2.imread(r'D:\ML DEEP LEARNING FACE\Masked Face Recognition\data\MaskImages\without_mask\0_0_caizhuoyan_0014.jpg')
# max = np.max(predict(img))
label, predictions = detect(img)
print(label)
print(np.max(predictions))

In [ ]:
cap = cv2.VideoCapture(0)
cascade_classifier = cv2.CascadeClassifier('D:\\ML DEEP LEARNING FACE\\Masked Face Recognition\\haarcascade_frontalface_default.xml')

while cap.isOpened():
    ret,frame=cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    detection = cascade_classifier.detectMultiScale(gray,1.3,5)
#     cv2.imshow('frame',frame)
    
    for d in detection:
        (x,y,w,h)=d
        frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        img_crop=frame[y:y+h,x:x+w]
        img_crop=cv2.resize(img_crop,(255,255))
        label, predictions = detect(img_crop)
        img = frame[y:y+h-120,x:x+w]
        name = predict(img)
        
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, np.max(predictions) * 100)

        cv2.putText(frame, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h-120), (0,0,0), 2)
        cv2.putText(frame, name, (x+w-20, y+h+10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()